In [1]:
%load_ext autoreload

%autoreload 2


In [2]:
import numpy
import pandas as pd
import seaborn as sns
import random
import math
import os

from IPython import display

pd.set_option('display.max_columns', None)

### Import model objects from their unique directories - easier to organize

from commuter_model.commuter_model import CommuterModel
from electrical_model.electrical_model import ElectricModel

In [3]:
# load ipums data to local variable (ipums_df_19)
ipums_df_19 = pd.read_csv("commuter_model/disaggregated_cleaned_ipums_data.csv",index_col=0)
ipums_df_19 = ipums_df_19[ipums_df_19['YEAR']==2019].reset_index(drop=True).copy()



In [36]:
for x in ipums_df_19.columns:
    print(x)

YEAR
PERWT
HOME_STATEFIP
HOME_PUMA
PUMA_NAME
SEX
AGE
HRS_WK_DAILY
TOTAL_PERSONAL_INCOME
MODE_TRANSP_TO_WORK
MODE_TRANSP_TO_WORK_HBDMATCH
HOMEOWNER_LABEL
RACE_LABEL
EDUC_LABEL
DEPARTS_FOR_WORK_HOUR
ARRIVES_AT_WORK_HOUR
COMMUTE_DIRECTION_MANHATTAN
VEHICLE_AVAILABLE
OCC_DESCRIPTION
IND_CAT
IND_DESCRIPTION
PUMAKEY_HOME
PUMAKEY_WORK
DISTANCE_KM
COGNITIVE_DIFFICULTY
AMBULATORY_DIFFICULTY
IND_LIVING_DIFFICULTY
SELFCARE_DIFFICULTY
VISION_OR_HEARING_DIFFICULTY
VISION_DIFFICULTY
HEARING_DIFFICULTY


In [4]:
commuter_model_alltransit_no_wfh = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_alltransit_heavy_wfh = CommuterModel(ipums_df=ipums_df_19.copy())


commuter_model_allmicro = CommuterModel(ipums_df=ipums_df_19.copy())
commuter_model_mix = CommuterModel(ipums_df=ipums_df_19.copy())

In [5]:
## All Transit - no WFH
commuter_model_alltransit_no_wfh.ipums_df['FLAG_AUTO'] = 0
commuter_model_alltransit_no_wfh.ipums_df['FLAG_ESCOOTER'] = 0
commuter_model_alltransit_no_wfh.ipums_df['FLAG_WALK'] = 0
commuter_model_alltransit_no_wfh.ipums_df['FLAG_EBIKE']= 0
commuter_model_alltransit_no_wfh.ipums_df['FLAG_MOTORCYCLE']= 0
commuter_model_alltransit_no_wfh.ipums_df['FLAG_TAXICAB']= 0

# print(commuter_model_alltransit.ipums_df[['FLAG_AUTO',"FLAG_ESCOOTER","FLAG_WALK","FLAG_EBIKE"]].sum())

#### Transit
### Bus
commuter_model_alltransit_no_wfh.ipums_df['FLAG_EBUSES'] = commuter_model_alltransit_no_wfh.bus_flag_binary(
                                        home_region=commuter_model_alltransit_no_wfh.puma_home_Bus
                                        ,work_region=commuter_model_alltransit_no_wfh.puma_work_Bus
                                        ,schedule=commuter_model_alltransit_no_wfh.time_Bus
                                        ,affordability=100
                                        ,fixgaps=False
                                         )

## Subway
commuter_model_alltransit_no_wfh.ipums_df['FLAG_SUBWAY'] = commuter_model_alltransit_no_wfh.subway_flag_binary(
                                            home_region=commuter_model_alltransit_no_wfh.puma_home_Subway
                                            ,work_region=commuter_model_alltransit_no_wfh.puma_work_Subway
                                            ,schedule=commuter_model_alltransit_no_wfh.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )


## Commuter Rail
commuter_model_alltransit_no_wfh.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_alltransit_no_wfh.commuterRail_flag_binary(
                                             home_region=commuter_model_alltransit_no_wfh.puma_home_CommuterRail
                                            ,work_region=commuter_model_alltransit_no_wfh.puma_work_CommuterRail
                                            ,schedule=commuter_model_alltransit_no_wfh.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_alltransit_no_wfh.ipums_df['FLAG_FERRY'] = commuter_model_alltransit_no_wfh.ferry_flag_binary(
                                            home_region=commuter_model_alltransit_no_wfh.puma_home_Ferry
                                           ,work_region=commuter_model_alltransit_no_wfh.puma_work_Ferry
                                           ,schedule=commuter_model_alltransit_no_wfh.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )
## Work From Home
commuter_model_alltransit_no_wfh.ipums_df["FLAG_WFH"] = 0

### Assign transit mode hierarchically
commuter_model_alltransit_no_wfh.ipums_df['First_Assignment'] = commuter_model_alltransit_no_wfh\
            .AssignTransModesWaterfall(["FLAG_SUBWAY","FLAG_EBUSES","FLAG_COMMUTERRAIL","FLAG_FERRY"])

commuter_model_alltransit_no_wfh.ipums_df['TransMode'] = commuter_model_alltransit_no_wfh.RandAssignChosenOption(
                                                                commuter_model_alltransit_no_wfh.ipums_df['First_Assignment'], 
                                                                commuter_model_alltransit_no_wfh.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [6]:
flag_cols = [x for x in commuter_model_alltransit_no_wfh.ipums_df if "FLAG" in x]


In [7]:
commuter_model_alltransit_no_wfh.ipums_df[flag_cols].sum()

FLAG_AUTO                0
FLAG_ESCOOTER            0
FLAG_WALK                0
FLAG_EBIKE               0
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_WFH                 0
dtype: int64

In [8]:
commuter_model_alltransit_no_wfh.ipums_df["First_Assignment"].value_counts()

Subway          17010
CommuterRail     2745
Bus              2511
No Option        1674
Ferry              38
Name: First_Assignment, dtype: int64

In [9]:
commuter_model_alltransit_no_wfh.ipums_df["TransMode"].value_counts()

Subway           17261
CommuterRail      3239
Bus               2804
AutoOccupants      521
Ferry               74
Walk                46
Other               21
Bicycle              7
WFH                  5
Name: TransMode, dtype: int64

In [10]:
commuter_model_alltransit_no_wfh.ipums_df[flag_cols].sum()

FLAG_AUTO                0
FLAG_ESCOOTER            0
FLAG_WALK                0
FLAG_EBIKE               0
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_WFH                 0
dtype: int64

In [11]:
### All Transit - Higher WFH
commuter_model_alltransit_heavy_wfh.ipums_df['FLAG_AUTO'] = 0
commuter_model_alltransit_heavy_wfh.ipums_df['FLAG_ESCOOTER'] = 0
commuter_model_alltransit_heavy_wfh.ipums_df['FLAG_WALK'] = 0
commuter_model_alltransit_heavy_wfh.ipums_df['FLAG_EBIKE']= 0
commuter_model_alltransit_heavy_wfh.ipums_df['FLAG_MOTORCYCLE']= 0
commuter_model_alltransit_heavy_wfh.ipums_df['FLAG_TAXICAB']= 0

# print(commuter_model_alltransit.ipums_df[['FLAG_AUTO',"FLAG_ESCOOTER","FLAG_WALK","FLAG_EBIKE"]].sum())

#### Transit
### Bus
commuter_model_alltransit_heavy_wfh.ipums_df['FLAG_EBUSES'] = commuter_model_alltransit_heavy_wfh.bus_flag_binary(
                                        home_region=commuter_model_alltransit_heavy_wfh.puma_home_Bus
                                        ,work_region=commuter_model_alltransit_heavy_wfh.puma_work_Bus
                                        ,schedule=commuter_model_alltransit_heavy_wfh.time_Bus
                                        ,affordability=100
                                        ,fixgaps=False
                                         )

## Subway
commuter_model_alltransit_heavy_wfh.ipums_df['FLAG_SUBWAY'] = commuter_model_alltransit_heavy_wfh.subway_flag_binary(
                                            home_region=commuter_model_alltransit_heavy_wfh.puma_home_Subway
                                            ,work_region=commuter_model_alltransit_heavy_wfh.puma_work_Subway
                                            ,schedule=commuter_model_alltransit_heavy_wfh.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )


## Commuter Rail
commuter_model_alltransit_heavy_wfh.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_alltransit_heavy_wfh.commuterRail_flag_binary(
                                             home_region=commuter_model_alltransit_heavy_wfh.puma_home_CommuterRail
                                            ,work_region=commuter_model_alltransit_heavy_wfh.puma_work_CommuterRail
                                            ,schedule=commuter_model_alltransit_heavy_wfh.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_alltransit_heavy_wfh.ipums_df['FLAG_FERRY'] = commuter_model_alltransit_heavy_wfh.ferry_flag_binary(
                                            home_region=commuter_model_alltransit_heavy_wfh.puma_home_Ferry
                                           ,work_region=commuter_model_alltransit_heavy_wfh.puma_work_Ferry
                                           ,schedule=commuter_model_alltransit_heavy_wfh.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )
## Work From Home
commuter_model_alltransit_heavy_wfh.ipums_df["FLAG_WFH"] = commuter_model_alltransit_heavy_wfh.wfh_flag_binary(
                                            wfh_dampener=1)

# print(commuter_model_alltransit.ipums_df[flag_cols].sum())

### Assign transit mode hierarchically
commuter_model_alltransit_heavy_wfh.ipums_df['First_Assignment'] = commuter_model_alltransit_heavy_wfh\
            .AssignTransModesWaterfall(["FLAG_WFH","FLAG_SUBWAY","FLAG_EBUSES","FLAG_COMMUTERRAIL","FLAG_FERRY"])

commuter_model_alltransit_heavy_wfh.ipums_df['TransMode'] = commuter_model_alltransit_heavy_wfh.RandAssignChosenOption(
                                                                commuter_model_alltransit_heavy_wfh.ipums_df['First_Assignment'], 
                                                                commuter_model_alltransit_heavy_wfh.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [12]:
commuter_model_alltransit_heavy_wfh.ipums_df[flag_cols].sum()

FLAG_AUTO                0
FLAG_ESCOOTER            0
FLAG_WALK                0
FLAG_EBIKE               0
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_WFH              6526
dtype: int64

In [13]:
commuter_model_alltransit_heavy_wfh.ipums_df["First_Assignment"].value_counts()

Subway          12697
WFH              6526
Bus              1845
CommuterRail     1587
No Option        1293
Ferry              30
Name: First_Assignment, dtype: int64

In [14]:
commuter_model_alltransit_heavy_wfh.ipums_df["TransMode"].value_counts()

Subway           12890
WFH               6531
Bus               2084
CommuterRail      1869
AutoOccupants      474
Ferry               66
Walk                40
Other               17
Bicycle              7
Name: TransMode, dtype: int64

In [15]:
commuter_model_alltransit_heavy_wfh.ipums_df[flag_cols].sum()

FLAG_AUTO                0
FLAG_ESCOOTER            0
FLAG_WALK                0
FLAG_EBIKE               0
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES          13958
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_WFH              6526
dtype: int64

In [16]:
#### All Micro-Mobility
commuter_model_allmicro.ipums_df['FLAG_AUTO'] = 0
## Motorcycle
commuter_model_allmicro.ipums_df['FLAG_MOTORCYCLE'] = 0
## Taxicab
commuter_model_allmicro.ipums_df['FLAG_TAXICAB'] = 0
## Bus
commuter_model_allmicro.ipums_df['FLAG_EBUSES'] = 0
## Subway
commuter_model_allmicro.ipums_df['FLAG_SUBWAY'] = 0
## Commuter Rail
commuter_model_allmicro.ipums_df['FLAG_COMMUTERRAIL'] = 0
## Ferry
commuter_model_allmicro.ipums_df['FLAG_FERRY'] = 0


## E-Scooter - bump up distance
commuter_model_allmicro.ipums_df['FLAG_ESCOOTER'] = commuter_model_allmicro.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=15
                                            ,scooter_friendly_origins=commuter_model_allmicro.bike_friendly_origins
                                            ,male_pct=100
                                            ,female_pct=100
                                            )


## Walking
commuter_model_allmicro.ipums_df['FLAG_WALK'] = commuter_model_allmicro.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_allmicro.ipums_df['FLAG_EBIKE']=commuter_model_allmicro.ebike_flag_binary(
                                        max_age=100
                                        ,max_distance = 24
                                        ,male_pct = 100
                                        ,female_pct = 100
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_allmicro.bike_friendly_origins
                                        )


## Work From Home
commuter_model_allmicro.ipums_df["FLAG_WFH"] = commuter_model_allmicro.wfh_flag_binary(
                                            wfh_dampener=1
                                                )
### Assign transit mode hierarchically
commuter_model_allmicro.ipums_df['First_Assignment'] = commuter_model_allmicro.AssignTransModesWaterfall(
                                                    ["FLAG_EBIKE","FLAG_ESCOOTER","FLAG_WALK","FLAG_WFH"]
                                                                                )

commuter_model_allmicro.ipums_df['TransMode'] = commuter_model_allmicro.RandAssignChosenOption(
                                                                commuter_model_allmicro.ipums_df['First_Assignment'], 
                                                                commuter_model_allmicro.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [17]:
commuter_model_allmicro.ipums_df[flag_cols].sum()

FLAG_AUTO                0
FLAG_ESCOOTER         8649
FLAG_WALK             1082
FLAG_EBIKE           14406
FLAG_MOTORCYCLE          0
FLAG_TAXICAB             0
FLAG_EBUSES              0
FLAG_SUBWAY              0
FLAG_COMMUTERRAIL        0
FLAG_FERRY               0
FLAG_WFH              6524
dtype: int64

In [18]:
commuter_model_allmicro.ipums_df["First_Assignment"].value_counts(dropna=False)

Bicycle      14406
No Option     6650
WFH           2922
Name: First_Assignment, dtype: int64

In [19]:
commuter_model_allmicro.ipums_df["TransMode"].value_counts()

Bicycle          14424
WFH               2922
AutoOccupants     1747
Subway            1724
CommuterRail      1662
Bus               1213
Ferry              189
Walk                69
Other               28
Name: TransMode, dtype: int64

In [20]:
#### Standard

commuter_model_mix.ipums_df['FLAG_AUTO'] = commuter_model_mix.auto_flag_binary(
                                        max_age=70
                                       ,min_distance = 2
                                       ,min_income = 32626
                                       ,male_pct = 15
                                       ,female_pct = 15
                                       ,age_dist = None
                                       )
## Motorcycle
commuter_model_mix.ipums_df['FLAG_MOTORCYCLE'] = commuter_model_mix.motorcycle_flag_binary(
                                        max_age=35
                                       ,max_distance = 300
                                       ,min_income = 50000
                                       ,male_pct = 1
                                       ,female_pct = 0
                                       ,age_dist = None
                                        )


## Taxicab
commuter_model_mix.ipums_df['FLAG_TAXICAB'] = commuter_model_mix.taxicab_flag_binary(
                                        max_distance = 15
                                       ,min_income = 32626
                                       ,male_pct = 5
                                       ,female_pct = 2
                                       ,age_dist = None
                                        )

## Bus
commuter_model_mix.ipums_df['FLAG_EBUSES'] = commuter_model_mix.bus_flag_binary(
                                        home_region=commuter_model_mix.puma_home_Bus
                                        ,work_region=commuter_model_mix.puma_work_Bus
                                        ,schedule=commuter_model_mix.time_Bus
                                        ,affordability=50
                                        ,fixgaps=False
                                         )


## Subway
commuter_model_mix.ipums_df['FLAG_SUBWAY'] = commuter_model_mix.subway_flag_binary(
                                            home_region=commuter_model_mix.puma_home_Subway
                                            ,work_region=commuter_model_mix.puma_work_Subway
                                            ,schedule=commuter_model_mix.time_Subway
                                            ,affordability=100
                                            ,fixgaps=False
                                            )



## Commuter Rail
commuter_model_mix.ipums_df['FLAG_COMMUTERRAIL'] = commuter_model_mix.commuterRail_flag_binary(
                                             home_region=commuter_model_mix.puma_home_CommuterRail
                                            ,work_region=commuter_model_mix.puma_work_CommuterRail
                                            ,schedule=commuter_model_mix.time_CommuterRail
                                            ,affordability=100
                                            ,fixgaps=False
                                            )

## Ferry
commuter_model_mix.ipums_df['FLAG_FERRY'] = commuter_model_mix.ferry_flag_binary(
                                            home_region=commuter_model_mix.puma_home_Ferry
                                           ,work_region=commuter_model_mix.puma_work_Ferry
                                           ,schedule=commuter_model_mix.time_Ferry
                                           ,affordability=100
                                           ,fixgaps=False
                                          )



## E-Scooter
commuter_model_mix.ipums_df['FLAG_ESCOOTER'] = commuter_model_mix.escooter_flag_binary(
                                            max_age=60
                                            ,max_distance=4
                                            ,scooter_friendly_origins=commuter_model_mix.bike_friendly_origins
                                            ,male_pct=25
                                            ,female_pct=25
                                            )


## Walking
commuter_model_mix.ipums_df['FLAG_WALK'] = commuter_model_mix.walking_flag_binary(
                                            max_age=100
                                            ,max_distance=2
                                            ,male_pct=100
                                            ,female_pct=100
                                            )

## E-Bike
commuter_model_mix.ipums_df['FLAG_EBIKE']=commuter_model_mix.ebike_flag_binary(
                                        max_age=70
                                        ,max_distance = 35
                                        ,male_pct = 50
                                        ,female_pct = 40
                                        ,age_dist = None
                                        ,bike_friendly_origins=commuter_model_mix.bike_friendly_origins
                                        )


## Work From Home
commuter_model_mix.ipums_df["FLAG_WFH"] = commuter_model_mix.wfh_flag_binary(
                                            wfh_dampener=.5
                                                )

## Keep random assignment for mixed transit option
commuter_model_mix.ipums_df['First_Assignment'] = commuter_model_mix.RandAssignNoOption(
                                                                commuter_model_mix.ipums_df['FLAG_AUTO'], 
                                                                commuter_model_mix.ipums_df['FLAG_MOTORCYCLE'],
                                                                commuter_model_mix.ipums_df['FLAG_TAXICAB'], 
                                                                commuter_model_mix.ipums_df['FLAG_EBUSES'],
                                                                commuter_model_mix.ipums_df['FLAG_SUBWAY'], 
                                                                commuter_model_mix.ipums_df['FLAG_COMMUTERRAIL'],
                                                                commuter_model_mix.ipums_df['FLAG_FERRY'], 
                                                                commuter_model_mix.ipums_df['FLAG_ESCOOTER'],
                                                                commuter_model_mix.ipums_df['FLAG_WALK'], 
                                                                commuter_model_mix.ipums_df['FLAG_EBIKE'],
                                                                commuter_model_mix.ipums_df['FLAG_WFH'])
commuter_model_mix.ipums_df['TransMode'] = commuter_model_mix.RandAssignChosenOption(
                                                                commuter_model_mix.ipums_df['First_Assignment'], 
                                                                commuter_model_mix.ipums_df['MODE_TRANSP_TO_WORK_HBDMATCH'],
                                                                "Current")

In [21]:
commuter_model_mix.ipums_df[flag_cols].sum()

FLAG_AUTO             1680
FLAG_ESCOOTER          232
FLAG_WALK             1082
FLAG_EBIKE            7315
FLAG_MOTORCYCLE         35
FLAG_TAXICAB           279
FLAG_EBUSES          13866
FLAG_SUBWAY          17010
FLAG_COMMUTERRAIL     7484
FLAG_FERRY            2516
FLAG_WFH              3223
dtype: int64

In [22]:
commuter_model_mix.ipums_df["First_Assignment"].value_counts()

Subway           7650
Bus              5331
CommuterRail     3915
Bicycle          2599
WFH              1260
No Option        1225
Ferry             869
AutoOccupants     694
Walk              301
Taxicab            77
Escooter           48
Motorcycle          9
Name: First_Assignment, dtype: int64

In [23]:
commuter_model_mix.ipums_df["TransMode"].value_counts()

Subway           7814
Bus              5560
CommuterRail     4244
Bicycle          2605
WFH              1260
AutoOccupants    1113
Ferry             899
Walk              333
Taxicab            77
Escooter           48
Other              16
Motorcycle          9
Name: TransMode, dtype: int64

In [24]:
#### Compare Counts of Flags
# commuter_model_alltransit.ipums_df['FLAG_AUTO'].sum()
ct_1 = commuter_model_allmicro.ipums_df['First_Assignment'].value_counts(dropna=False)

# commuter_model_alltransit.ipums_df['FLAG_AUTO'].sum()
ct_2 = commuter_model_alltransit_heavy_wfh.ipums_df['First_Assignment'].value_counts(dropna=False)

# commuter_model_alltransit.ipums_df['FLAG_AUTO'].sum()
ct_4 = commuter_model_alltransit_no_wfh.ipums_df['First_Assignment'].value_counts(dropna=False)

# commuter_model_mix.ipums_df['FLAG_AUTO'].sum()
ct_3 =commuter_model_mix.ipums_df['First_Assignment'].value_counts(dropna=False)

compare_cts = pd.merge(left=ct_1,right=ct_2,left_index=True,right_index=True,how='outer')\
.merge(right=ct_3,left_index=True,right_index=True,how='outer')\
.merge(right=ct_4,left_index=True,right_index=True,how='outer').fillna(0)

compare_cts.columns = ["All Micromobility","All Transit - Heavy WFH","All Transit - No WFH","Mixed All"]
compare_cts
# commuter_model_allmicro.ipums_df['First_Assignment'].value_counts(dropna=False)

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'First_Assignment_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


,All Micromobility,All Transit - Heavy WFH,All Transit - No WFH,Mixed All
AutoOccupants,0.0,0.0,694,0.0
Bicycle,14406.0,0.0,2599,0.0
Bus,0.0,1845.0,5331,2511.0
CommuterRail,0.0,1587.0,3915,2745.0
Escooter,0.0,0.0,48,0.0
Ferry,0.0,30.0,869,38.0
Motorcycle,0.0,0.0,9,0.0
No Option,6650.0,1293.0,1225,1674.0
Subway,0.0,12697.0,7650,17010.0
Taxicab,0.0,0.0,77,0.0


In [35]:
for x in commuter_model_mix.ipums_df.columns:
    print(x)
commuter_model_mix.ipums_df.head(2)

YEAR
PERWT
HOME_STATEFIP
HOME_PUMA
PUMA_NAME
SEX
AGE
HRS_WK_DAILY
TOTAL_PERSONAL_INCOME
MODE_TRANSP_TO_WORK
MODE_TRANSP_TO_WORK_HBDMATCH
HOMEOWNER_LABEL
RACE_LABEL
EDUC_LABEL
DEPARTS_FOR_WORK_HOUR
ARRIVES_AT_WORK_HOUR
COMMUTE_DIRECTION_MANHATTAN
VEHICLE_AVAILABLE
OCC_DESCRIPTION
IND_CAT
IND_DESCRIPTION
PUMAKEY_HOME
PUMAKEY_WORK
DISTANCE_KM
COGNITIVE_DIFFICULTY
AMBULATORY_DIFFICULTY
IND_LIVING_DIFFICULTY
SELFCARE_DIFFICULTY
VISION_OR_HEARING_DIFFICULTY
VISION_DIFFICULTY
HEARING_DIFFICULTY
FLAG_AUTO
FLAG_MOTORCYCLE
FLAG_TAXICAB
FLAG_EBUSES
FLAG_SUBWAY
FLAG_COMMUTERRAIL
FLAG_FERRY
FLAG_ESCOOTER
FLAG_WALK
FLAG_EBIKE
FLAG_WFH
First_Assignment
TransMode
MNY_RES


,YEAR,PERWT,HOME_STATEFIP,HOME_PUMA,PUMA_NAME,SEX,AGE,HRS_WK_DAILY,TOTAL_PERSONAL_INCOME,MODE_TRANSP_TO_WORK,MODE_TRANSP_TO_WORK_HBDMATCH,HOMEOWNER_LABEL,RACE_LABEL,EDUC_LABEL,DEPARTS_FOR_WORK_HOUR,ARRIVES_AT_WORK_HOUR,COMMUTE_DIRECTION_MANHATTAN,VEHICLE_AVAILABLE,OCC_DESCRIPTION,IND_CAT,IND_DESCRIPTION,PUMAKEY_HOME,PUMAKEY_WORK,DISTANCE_KM,COGNITIVE_DIFFICULTY,AMBULATORY_DIFFICULTY,IND_LIVING_DIFFICULTY,SELFCARE_DIFFICULTY,VISION_OR_HEARING_DIFFICULTY,VISION_DIFFICULTY,HEARING_DIFFICULTY,FLAG_AUTO,FLAG_MOTORCYCLE,FLAG_TAXICAB,FLAG_EBUSES,FLAG_SUBWAY,FLAG_COMMUTERRAIL,FLAG_FERRY,FLAG_ESCOOTER,FLAG_WALK,FLAG_EBIKE,FLAG_WFH,First_Assignment,TransMode,MNY_RES
0,2019,23.0,9,500,Litchfield County,F,46,7.4,3800,"Auto, truck, or van",AutoOccupants,Rent,White,Grades12,8,9,in,1,Personal care aides,"Educational Services, and Health Care and Soci...",Home health care services,09_00500,36_03800,170.50491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,No Option,AutoOccupants,0
1,2019,48.0,9,500,Litchfield County,M,51,11.0,488000,"Auto, truck, or van",AutoOccupants,Rent,White,College_5PlusYears,8,8,in,1,"Lawyers, and judges, magistrates, and other ju...","Professional, Scientific, and Management, and ...",Legal services,09_00500,36_03800,170.50491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,No Option,AutoOccupants,0


In [25]:
master_df = pd.DataFrame()
for x in [commuter_model_allmicro
        ,commuter_model_alltransit_heavy_wfh
        ,commuter_model_alltransit_no_wfh
        ,commuter_model_mix]:
    master_df = pd.concat([master_df,x.ipums_df.groupby(by=["TransMode"])['PERWT'].sum()],axis=1)
master_df.columns=["all_micro_count","all_transit_heavyWFH_count","all_transit_noWFH_count","mix_count"]
master_df

,all_micro_count,all_transit_heavyWFH_count,all_transit_noWFH_count,mix_count
AutoOccupants,185968.0,49330.0,54010.0,113936.0
Bicycle,1852839.0,540.0,540.0,319204.0
Bus,134656.0,224174.0,291568.0,689463.0
CommuterRail,165053.0,181721.0,304901.0,429190.0
Ferry,19307.0,5592.0,6068.0,92284.0
Other,2802.0,1995.0,2422.0,1879.0
Subway,161061.0,1625328.0,2136579.0,947691.0
WFH,273082.0,708253.0,463.0,131779.0
Walk,6562.0,4397.0,4779.0,54945.0
Escooter,NaN,NaN,NaN,8071.0


In [26]:
### Electric Model
### read in commuter_df. feed that and the ev file location to class
em_1 = ElectricModel(commuter_model_alltransit_heavy_wfh.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'alltransit_heavywfh')
em_2 = ElectricModel(commuter_model_allmicro.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'allmicro')
em_3 = ElectricModel(commuter_model_mix.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                    ,model_name = 'allmodes')
em_4 = ElectricModel(commuter_model_alltransit_no_wfh.ipums_df
                     ,ev_reference_table_loc="electrical_model/EV_reference_table.csv"\
                    ,PEV_delay_hr=3
                     ,bus_ferry_cab_delay=6
                     ,model_name = 'alltransit_nowfh')


In [27]:
ct= 0 
for e_m in [em_1,em_2,em_3,em_4]:
    ct+=1
    ## Run methods in order to read in, clean, and aggregate data
    e_m.read_electric_info_in()

    e_m.clean_commuter_df()

    e_m.create_in_out_flow_dfs()

    e_m.aggregate_data()
    print(f"electric model done for em {ct}: {e_m.model_name}")

ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 1: alltransit_heavywfh
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 2: allmicro
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 3: allmodes
ev reference table read in. subway floor and commuter rail floor extracted from table
electric model done for em 4: alltransit_nowfh


In [30]:
print(em_1.df_aggregate.columns)
em_1.df_aggregate.head(10)

Index(['Charge_Hour', 'TransMode', 'PEV_DELAY', 'FLOW_DIR', 'Energy'], dtype='object')


,Charge_Hour,TransMode,PEV_DELAY,FLOW_DIR,Energy
0,0,AutoOccupants,Custom,IN,3421.942918
1,0,AutoOccupants,Earliest,IN,11530.679025
2,0,AutoOccupants,Latest,IN,2438.541304
3,0,AutoOccupants,Random,IN,7434.963423
4,0,CommuterRail,Custom,IN,9171.480000
5,0,CommuterRail,Custom,OUT,9633.720000
6,0,CommuterRail,Earliest,IN,9171.480000
7,0,CommuterRail,Earliest,OUT,9633.720000
8,0,CommuterRail,Latest,IN,9171.480000
9,0,CommuterRail,Latest,OUT,9633.720000


In [28]:
breakp = 

SyntaxError: invalid syntax (576900331.py, line 1)

In [ ]:
save_dir = "../06_Dashboard_Visualization/electric_model_outputs"
os.makedirs(save_dir,exist_ok=True)
for e_m in [em_1,em_2,em_3,em_4]:
    # Save in save_dir
    e_m.df_aggregate.to_pickle(f"{save_dir}/electric_model_{e_m.model_name}_dfaggregate.pkl")